In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/CelebA.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/CelebA_Data')

In [ ]:
with zipfile.ZipFile('/content/CelebA_Data/CelebA/Img/img_align_celeba.zip','r') as zip_ref_img:
  zip_ref_img.extractall('/content/')

In [ ]:
import zipfile
import random
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import Dataset
import PIL
import os
import cv2
from torch import optim
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn as nn
from torch.autograd import Variable
from torchsummary import summary
import matplotlib.pyplot as plt


import torchvision.transforms as transforms

from torch.utils.data import Dataset

import PIL
from PIL import ImageChops
from sklearn.metrics import accuracy_score
from matplotlib import image as img

In [ ]:
img_dir = '/content/celebImages/img_align_celeba'
transform = transforms.Compose([transforms.Grayscale(), 
                                         transforms.ToTensor(),
                                         transforms.Resize((28,28)), 
                                         transforms.Normalize(0,0.5)])
label_file = '/content/CelebA_Data/CelebA/Anno/identity_CelebA.txt'
batch_size = 4

In [ ]:
df = pd.read_csv(label_file, sep=" ", header=None)
df.columns = ["name", "ID"]
df.sort_values(by = "ID", ignore_index=True)

,name,ID
0,056279.jpg,1
1,145590.jpg,1
2,068169.jpg,1
3,000023.jpg,1
4,024091.jpg,1
...,...,...
202594,130636.jpg,10177
202595,129750.jpg,10177
202596,136615.jpg,10177
202597,053390.jpg,10177


In [ ]:
sub_df = df.loc[0:1499]
sub_df

,name,ID
0,000001.jpg,2880
1,000002.jpg,2937
2,000003.jpg,8692
3,000004.jpg,5805
4,000005.jpg,9295
...,...,...
1495,001496.jpg,4648
1496,001497.jpg,3431
1497,001498.jpg,5715
1498,001499.jpg,7025


In [ ]:
sub_df['key'] = 1
combined_df = pd.merge(sub_df, sub_df, on = 'key')
del combined_df['key']
combined_df.to_csv('/content/drive/MyDrive/task2_data.csv', index = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
print(combined_df)

             name_x  ID_x      name_y  ID_y
0        000001.jpg  2880  000001.jpg  2880
1        000001.jpg  2880  000002.jpg  2937
2        000001.jpg  2880  000003.jpg  8692
3        000001.jpg  2880  000004.jpg  5805
4        000001.jpg  2880  000005.jpg  9295
...             ...   ...         ...   ...
2249995  001500.jpg  4027  001496.jpg  4648
2249996  001500.jpg  4027  001497.jpg  3431
2249997  001500.jpg  4027  001498.jpg  5715
2249998  001500.jpg  4027  001499.jpg  7025
2249999  001500.jpg  4027  001500.jpg  4027

[2250000 rows x 4 columns]


In [ ]:
def match(row):
    if row['ID_x'] == row['ID_y']:
        return 1
    else:
        return 0

In [ ]:
labeled_df = pd.DataFrame()
labeled_df[['Image1', 'Image2']] = combined_df[['name_x', 'name_y']]
labeled_df

,Image1,Image2
0,000001.jpg,000001.jpg
1,000001.jpg,000002.jpg
2,000001.jpg,000003.jpg
3,000001.jpg,000004.jpg
4,000001.jpg,000005.jpg
...,...,...
2249995,001500.jpg,001496.jpg
2249996,001500.jpg,001497.jpg
2249997,001500.jpg,001498.jpg
2249998,001500.jpg,001499.jpg


In [ ]:
labeled_df['label'] = combined_df.apply(match, axis = 1)
labeled_df.to_csv('/content/drive/MyDrive/task2_labeleddata.csv', index = False)

In [ ]:
values = labeled_df['label'].value_counts()
print(values)
difference = values[0] - values[1]

0    2248152
1       1848
Name: label, dtype: int64


In [ ]:
labeled_df = labeled_df.sort_values(by = "label", ignore_index=True)
bal_df = labeled_df[~labeled_df.index.isin(range(difference))]
bal_df['label'].value_counts()

0    1848
1    1848
Name: label, dtype: int64

In [ ]:
bal_file='/content/drive/MyDrive/task2_balanceddata.csv'
bal_df.to_csv(bal_file, index = False)
bal_df = pd.read_csv(bal_file)
bal_df = bal_df.sample(frac=1).reset_index(drop=True)#shuffling
print(bal_df)
bal_df.to_csv(bal_file, index = False)#putting it in csv file

          Image1      Image2  label
0     000887.jpg  000887.jpg      1
1     000030.jpg  000030.jpg      1
2     000501.jpg  000611.jpg      0
3     000500.jpg  001143.jpg      0
4     000626.jpg  000626.jpg      1
...          ...         ...    ...
3691  001485.jpg  001485.jpg      1
3692  000846.jpg  000846.jpg      1
3693  000501.jpg  000812.jpg      0
3694  000501.jpg  000013.jpg      0
3695  000501.jpg  000814.jpg      0

[3696 rows x 3 columns]


In [ ]:
from torch.utils.data import Dataset
import PIL
from PIL import ImageChops
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, data_dir, transform=None,target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.transform = transform
        self.target_transform = target_transform
        self.data_dir = data_dir

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path1 = self.data_dir + self.img_labels.iloc[idx, 0]
        img_path2 = self.data_dir + self.img_labels.iloc[idx, 1]
        
        
        image1 = PIL.Image.open(img_path1)
        image2 = PIL.Image.open(img_path2)
        
        diff = ImageChops.difference(image1, image2)
        
        label = torch.tensor(self.img_labels.iloc[idx, 2])
        
        if self.transform:
            diff = self.transform(diff)
            
        if self.target_transform:
            label = self.target_transform(label)
            
        return diff, label

In [ ]:
trans = transforms.Compose([transforms.Grayscale(1), transforms.Resize((40,40), transforms.InterpolationMode.BICUBIC), transforms.ToTensor(), transforms.Normalize([0.5],[0.5]),transforms.RandomHorizontalFlip()])

Mydataset=CustomImageDataset(bal_file,'/content/img_align_celeba/',transform=trans)
Total_Train_Size = int(len(Mydataset) * 0.7)
train_size = int((Total_Train_Size) * 0.7)+1
test_size = int(len(Mydataset) * 0.3)+1
validation_size = int(Total_Train_Size * 0.3 )
print(train_size+test_size+validation_size)
print(len(Mydataset))
initial_training_set, test_set = torch.utils.data.random_split(Mydataset, [Total_Train_Size, test_size])
train_set, validation_set = torch.utils.data.random_split(initial_training_set, [train_size, validation_size])

3696
3696


In [ ]:
print(validation_size)

776


In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=4,shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset=validation_set,batch_size=4,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=4,shuffle=True) 

In [ ]:
print(validation_size)

776


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1, # gray-scale images           
                out_channels=16,            
                kernel_size=5, # 5x5 convolutional kernel               
                stride=1,  #no. of pixels pass at a time                 
                padding=2, # to preserve size of input image                 
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layers
        self.out = nn.Linear(3200, 1)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # flatten the output of conv2 
        x = x.view(x.size(0), -1)    
        output = self.out(x)
        return output
cnn=CNN()
cnn
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=3200, out_features=1, bias=True)
)


In [ ]:
from torch import optim

# loss
criterion = nn.BCELoss()

#optimizer
optimizer = optim.Adam(cnn.parameters(), lr = 0.01) 

In [ ]:
from torch.autograd import Variable

epochsno = 4
#def train(num_epochs, cnn, loaders):
        
# Train the model
totalsteps = len(train_loader)
total_loss=[]
valid_loss=[]

for epoch in range(epochsno):
  epoch_loss = 0
  for i, (images, labels) in enumerate(validation_loader):
              # Forward pass
    outputs = cnn(images)
    outputs = torch.squeeze(outputs)
    #print(outputs)
    loss = criterion(outputs, labels.float())

      # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()   
    epoch_loss+=loss.item()
  valid_loss.append(epoch_loss)
  
  for i, (images, labels) in enumerate(train_loader):
              # Forward pass
    outputs = cnn(images)
    outputs = torch.squeeze(outputs)
    loss = criterion(outputs, labels.float())
      # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()  
    epoch_loss+=loss.item()
  
    if (i+1) % 100 == 0:
          print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                  .format(epoch + 1, epochsno, i + 1, totalsteps,loss.item()))
  total_loss.append(epoch_loss)

tensor([-0.1691, -0.1691, -0.1365, -0.0996], grad_fn=<SqueezeBackward0>)


RuntimeError: ignored

In [ ]:
plt.plot(total_loss,color='red')
plt.plot(valid_loss,color='green')
plt.title('Epochs vs Loss')
plt.show()

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        outputs = cnn(images)
         # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item() 
acc = 100.0 * n_correct / n_samples
print(f'Accuracy on test images: {acc} %') 